In [ ]:
#!/usr/bin/env python

# Reading the Fannie Mae and Freddie Mac housing datasets, transforming them, reading in the 
# relevant columns to a dataframe, and uniting those datasets into one.

# Data is available here:
# https://www.fhfa.gov/DataTools/Downloads/Pages/Profiles-of-Mortgage-Purchases-by-Fannie-Mae-and-Freddie-Mac.aspx


# sc = SparkContext()

import findspark
findspark.init()

import re
import sys
import spark
from pyspark import SparkContext
from pyspark.sql.types import *
import os
import sys

from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .master('local') \
        .appName("BroadbandScout")\
        .getOrCreate()

sc = SparkContext.getOrCreate()

#!/usr/bin/env python

# Reading the Fannie Mae and Freddie Mac housing datasets, transforming them, reading in the 
# relevant columns to a dataframe, and uniting those datasets into one.

# Data is available here: 


from pyspark.sql.functions import regexp_replace, col


pathname_FANNIE = 's3a://sparkforinsightproject/fnma_sf2017c_loans.txt'


pathname_FREDDIE = 's3a://sparkforinsightproject/fhlmc_sf2017c_loans.txt'


pathname_output_UNITED = 's3a://sparkforinsightproject/database_data/transformed_fhlmc_fnma_HOUSING_dataset'


def etl_fannie_freddie_data(input_FANNIE_txt, input_FREDDIE_txt):
    
    rdd_FANNIE_data = sc.textFile(input_FANNIE_txt)


    rdd_FANNIE_data = rdd_FANNIE_data.map(lambda x: x.encode('ascii', 'ignore'))\
                            .map(lambda l: l.split())\
                            .map(lambda l: (l[2] + l[4] + l[5], int(l[13]), l[36]))    

    df_FANNIE_data = spark.read.csv(rdd_FANNIE_data, mode='DROPMALFORMED', nullValue='NA', header=False)\
                                    .withColumn("_c0",regexp_replace(col("_c0"), "\(", ""))\
                                    .withColumn("_c2",regexp_replace(col("_c2"), "\)", ""))

        #, schema=housing_schema
    rdd_FREDDIE_data = sc.textFile(input_FREDDIE_txt)

    rdd_FREDDIE_data = rdd_FREDDIE_data.map(lambda x: x.encode('ascii', 'ignore'))\
                                    .map(lambda l: l.split())\
                                    .map(lambda l: (l[2] + l[4] + l[5], int(l[13]), l[36]))

    df_FREDDIE_data = spark.read.csv(rdd_FREDDIE_data, mode='DROPMALFORMED', nullValue='NA', header=False)\
                                    .withColumn("_c0",regexp_replace(col("_c0"), "\(", ""))\
                                    .withColumn("_c2",regexp_replace(col("_c2"), "\)", ""))

    
    df_FRANNIE_FREDDIE_UNITED = df_FANNIE_data.union(df_FREDDIE_data)

    df_FRANNIE_FREDDIE_UNITED = df_FRANNIE_FREDDIE_UNITED


    new_schema = ArrayType(IntegerType(), containsNull=True)


    return df_FRANNIE_FREDDIE_UNITED

def main():
    input_data_txt = sys.argv[1]
    output_data_txt = sys.argv[2]
    etl_fannie_freddie_data(input_FANNIE_txt, input_FREDDIE_txt)

    if __name__ == '__main__':
        main()

    
df_united_FANNIE_FREDDIE = etl_fannie_freddie_data(pathname_FANNIE, pathname_FREDDIE)


